<a href="https://colab.research.google.com/github/Vridhi-Wadhawan/deep-learning-models-in-practice/blob/main/01_forward_and_backprop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Network Forward & Backpropagation

## Overview

This notebook implements forward propagation and backpropagation from scratch for a simple 2–2–1 neural network.

What this notebook demonstrates:
- Xavier initialization
- Manual forward propagation
- Closed-form backpropagation gradients
- One-step gradient descent
- Effect of ReLU vs linear activation


## Initial Setup

In [ ]:
#Import Libraries
import numpy as np

In [ ]:
# Inputs
i1 = 2
i2 = 4
print("i1 = ", i1)
print("i2 = ", i2)

i1 =  2
i2 =  4


In [ ]:
# Setting random seed for reproducibility
np.random.seed(38)

## Xavier Initialization & Forward Propagation Using Linear Activation Function On Hidden Layers


**Why Xavier Initialization?**
- Prevents exploding/vanishing activations
- Keeps signal variance stable across layers

For a layer with 𝑛 inputs and 𝑛 outputs, weights are drawn from:

**𝑊∼𝑈(− sqrt.(6/𝑛 input + 𝑛 output), sqrt.(6/𝑛 input + 𝑛 output))**


In [ ]:
# Xavier initialization bounds
bound_input_hidden = np.sqrt(6 / (2 + 2))   # for W1, W2, W3 & W4
bound_hidden_output = np.sqrt(6 / (2 + 1)) # for W5 & W6

In [ ]:
# Initialize weights
W1, W2, W3, W4 = np.random.uniform(-bound_input_hidden, bound_input_hidden, 4)
W5, W6 = np.random.uniform(-bound_hidden_output, bound_hidden_output, 2)

initialized_weights = [W1, W2, W3, W4, W5, W6]

print("Initialized Weights:")
for idx, w in enumerate(initialized_weights, start=1):
    print(f"W{idx}: {w}")

Initialized Weights:
W1: -0.2822470618366393
W2: 0.8811006806223465
W3: 1.0880624516582986
W4: 0.49681749648057094
W5: 0.3779743052305753
W6: 0.2997037641550875


In [ ]:
# Forward propagation
h1 = W1 * i1 + W2 * i2 # hidden layer
h2 = W3 * i1 + W4 * i2 # hidden layer
out = W5 * h1 + W6 * h2 # output layer

print("\nForward Pass Results:")
print(f"h1 = {h1}, h2 = {h2}")
print(f"Output = {out}")


Forward Pass Results:
h1 = 2.9599085988161074, h2 = 4.163394889238881
Output = 2.36655451615247


**Forward Propagation (Linear Activation)**

Hidden units compute weighted sums of inputs.
No non-linearity is applied at the hidden layer.

In [ ]:
# Sanity check: manual vs computed forward pass
assert np.isclose(h1, W1 * i1 + W2 * i2)
assert np.isclose(h2, W3 * i1 + W4 * i2)

Xavier initialization enables stable signal propagation, while the linear forward pass applies pure weighted sums without non-linearity.


## Backpropagation Gradients (Closed Form)

Closed-form gradients are derived analytically and implemented directly below.

Final Gradient Set:

*Output Layer*
- ∂𝐿/∂𝑊5 = (out − 𝑦).(ℎ1)
- ∂𝐿/∂𝑊6 = (out − 𝑦).(ℎ2)

*Hidden Layer*
- ∂𝐿/∂𝑊1 = (out − 𝑦).(𝑊5).(𝑖1)
- ∂𝐿/∂𝑊2 = (out − 𝑦).(𝑊5).(𝑖2)
- ∂𝐿/∂𝑊3 = (out − 𝑦).(𝑊6).(𝑖1)
- ∂𝐿/∂𝑊4 = (out − 𝑦).(𝑊6).(𝑖2)

## One-Step Gradient Descent Update

Hyperparameters:
- target 𝑦 = 1
- learning rate 𝜂 = 0.05

In [ ]:
# Learning rate and target (given)
eta = 0.05
y = 1

In [ ]:
# Recomputing forward pass to check if the values are correct
h1 = W1 * i1 + W2 * i2
h2 = W3 * i1 + W4 * i2
out = W5 * h1 + W6 * h2

print("\nForward Pass Results:")
print(f"h1 = {h1}, h2 = {h2}")
print(f"Output = {out}")


Forward Pass Results:
h1 = 2.9599085988161074, h2 = 4.163394889238881
Output = 2.36655451615247


**Loss Function**

Mean Squared Error (MSE):   L = ½ (y − ŷ)²

In [ ]:
# Loss gradient with respect to output
dL_dout = out - y
print(f"Differentiating the loss w.r.t. output (dL/dout) = {dL_dout}")

Differentiating the loss w.r.t. output (dL/dout) = 1.36655451615247


In [ ]:
# Gradients
dL_dW5 = dL_dout * h1
dL_dW6 = dL_dout * h2

dL_dh1 = dL_dout * W5
dL_dh2 = dL_dout * W6

dL_dW1 = dL_dh1 * i1
dL_dW2 = dL_dh1 * i2
dL_dW3 = dL_dh2 * i1
dL_dW4 = dL_dh2 * i2

# Collect gradients
grads = [dL_dW1, dL_dW2, dL_dW3, dL_dW4, dL_dW5, dL_dW6]

print("Gradients before update:")
for idx, g in enumerate(grads, start=1):
    print(f"dL/dW{idx} = {g}")

Gradients before update:
dL/dW1 = 1.0330449876048695
dL/dW2 = 2.066089975209739
dL/dW3 = 0.8191230648280591
dL/dW4 = 1.6382461296561182
dL/dW5 = 4.044876463110681
dL/dW6 = 5.689506088415505


**Updated rule**
- For any weight 𝑊:

𝑊(new) = 𝑊(old) − (𝜂)⋅(∂𝐿/∂𝑊)

In [ ]:
# Update weights
W1_new = W1 - eta * dL_dW1
W2_new = W2 - eta * dL_dW2
W3_new = W3 - eta * dL_dW3
W4_new = W4 - eta * dL_dW4
W5_new = W5 - eta * dL_dW5
W6_new = W6 - eta * dL_dW6

# Collect updated weights
updated_weights = [W1_new, W2_new, W3_new, W4_new, W5_new, W6_new]

print("\nUpdated Weights:")
for idx, w in enumerate(updated_weights, start=1):
    print(f"W{idx}: {w}")


Updated Weights:
W1: -0.33389931121688277
W2: 0.7777961818618595
W3: 1.0471062984168957
W4: 0.41490518999776504
W5: 0.17573048207504124
W6: 0.01522845973431225


In [ ]:
# Recompute forward pass with updated weights
h1_new = W1_new * i1 + W2_new * i2
h2_new = W3_new * i1 + W4_new * i2
out_new = W5_new * h1_new + W6_new * h2_new

# Compute new loss
loss_new = 0.5 * (y - out_new) ** 2

print("\nAfter Weight Update:")
print(f"h1_new = {h1_new}")
print(f"h2_new = {h2_new}")
print(f"Output_new = {out_new}")
print(f"New Loss = {loss_new}")


After Weight Update:
h1_new = 2.4433861050136727
h2_new = 3.7538333568248516
Output_new = 0.4865425182532355
New Loss = 0.13181929278086452


After one gradient descent step, the loss decreases, confirming that the network learns in the correct direction.

## Forward Propagation with ReLU Activation Function on Hidden Layers

In [ ]:
# Using same inputs and initial weights from previous Questions

# Pre-activations for hidden layer
z1 = W1 * i1 + W2 * i2
z2 = W3 * i1 + W4 * i2

# ReLU activations
h1 = np.maximum(0, z1)
h2 = np.maximum(0, z2)

# Output layer (linear activation)
out = W5 * h1 + W6 * h2

print("With ReLU on hidden units:")
print(f"z1 = {z1}, z2 = {z2}")
print(f"h1 (ReLU) = {h1}, h2 (ReLU) = {h2}")
print(f"Output = {out}")


With ReLU on hidden units:
z1 = 2.9599085988161074, z2 = 4.163394889238881
h1 (ReLU) = 2.9599085988161074, h2 (ReLU) = 4.163394889238881
Output = 2.36655451615247


Since both z1 and z2 are positive, ReLU behaves identically to the linear activation in this specific example. If either one was not positive, then neuron’s activation becomes zero, removing its contribution to the output.

**Linear vs ReLU: Key Observation**

- Linear activation always propagates gradients
- ReLU blocks gradients when pre-activation ≤ 0
- In this example, both neurons remain active, so updates match the linear case


### Weight Updates with ReLU in Hidden Layer

In [ ]:
# Using same inputs and initial weights from previous Questions
# Forward pass (with ReLU on hidden)
z1 = W1 * i1 + W2 * i2
z2 = W3 * i1 + W4 * i2

h1 = np.maximum(0, z1)
h2 = np.maximum(0, z2)

out = W5 * h1 + W6 * h2

In [ ]:
# Compute loss
loss = 0.5 * (y - out) ** 2
print(f"Loss before update = {loss}")

Loss before update = 0.9337356228083555


In [ ]:
# Backpropagation
dL_dout = out - y
print(f"Differentiating the loss w.r.t. output (dL/dout) = {dL_dout}")

Differentiating the loss w.r.t. output (dL/dout) = 1.36655451615247


In [ ]:
# Gradients for W5, W6
dL_dW5 = dL_dout * h1
dL_dW6 = dL_dout * h2

In [ ]:
# Backprop to hidden layer (include ReLU derivative)
relu_grad_h1 = 1.0 if z1 > 0 else 0.0
relu_grad_h2 = 1.0 if z2 > 0 else 0.0

dL_dh1 = dL_dout * W5 * relu_grad_h1
dL_dh2 = dL_dout * W6 * relu_grad_h2

# Gradients for W1, W2, W3 & W4
dL_dW1 = dL_dh1 * i1
dL_dW2 = dL_dh1 * i2
dL_dW3 = dL_dh2 * i1
dL_dW4 = dL_dh2 * i2

# Collect gradients
grads = [dL_dW1, dL_dW2, dL_dW3, dL_dW4, dL_dW5, dL_dW6]

print("Gradients before update:")
for idx, g in enumerate(grads, start=1):
    print(f"dL/dW{idx} = {g}")

Gradients before update:
dL/dW1 = 1.0330449876048695
dL/dW2 = 2.066089975209739
dL/dW3 = 0.8191230648280591
dL/dW4 = 1.6382461296561182
dL/dW5 = 4.044876463110681
dL/dW6 = 5.689506088415505


In [ ]:
# Update weights
W1_new = W1 - eta * dL_dW1
W2_new = W2 - eta * dL_dW2
W3_new = W3 - eta * dL_dW3
W4_new = W4 - eta * dL_dW4
W5_new = W5 - eta * dL_dW5
W6_new = W6 - eta * dL_dW6

# Collect updated weights
updated_weights = [W1_new, W2_new, W3_new, W4_new, W5_new, W6_new]

print("\nUpdated Weights:")
for idx, w in enumerate(updated_weights, start=1):
    print(f"W{idx}: {w}")


Updated Weights:
W1: -0.33389931121688277
W2: 0.7777961818618595
W3: 1.0471062984168957
W4: 0.41490518999776504
W5: 0.17573048207504124
W6: 0.01522845973431225


In [ ]:
# Forward pass again with updated weights
h1_new = np.maximum(0, W1_new * i1 + W2_new * i2)
h2_new = np.maximum(0, W3_new * i1 + W4_new * i2)
out_new = W5_new * h1_new + W6_new * h2_new

loss_new = 0.5 * (y - out_new) ** 2

print("\nAfter Weight Update:")
print(f"h1_new = {h1_new}")
print(f"h2_new = {h2_new}")
print(f"Output_new = {out_new}")
print(f"New Loss = {loss_new}")


After Weight Update:
h1_new = 2.4433861050136727
h2_new = 3.7538333568248516
Output_new = 0.4865425182532355
New Loss = 0.13181929278086452


Since both z_1 and z_2 are positive, so ReLU doesn’t change their values. This makes h_1, h_2, the output, and the gradients identical to the linear activation case shown earlier. The loss decreases after the update, confirming learning. If either z =< 0, that neuron’s output would be 0 and its incoming weights would not update.

## Key Takeaways

- Backpropagation is a direct application of the chain rule
- Activation functions control gradient flow
- Weight initialization plays a critical role in stable learning

## Limitations

- Only a single training step is demonstrated
- No batching or multi-epoch optimization
- Network size is intentionally minimal for clarity